In [ ]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import time
import cv2
from timeit import default_timer as timer
from skimage.segmentation import felzenszwalb

from skimage import measure
from skimage.measure import find_contours, approximate_polygon,subdivide_polygon
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000


In [ ]:
def assemble(reservoir_shape ,pre_address,patchx,patchy,output_name):

    patches= np.sort(glob.glob(pre_address+'*'))
    print('number of patches: ',len(patches))

    mosaic = np.zeros(reservoir_shape).astype('uint8')
    for patch_name in patches :
        if True:
         #   print(patch_name)
            patch_img = cv2.imread(patch_name)

            numbers = []
            for word in patch_name.split():
                if word. isdigit():
                    numbers. append(int(word))

            line, col = numbers[0], numbers[1]

            y0, y1 = (col-1)*patchy, col*patchy
            if y1 > reservoir_shape[1]:
                y0, y1 = reservoir_shape[1]-patchy, reservoir_shape[1]

            x0,x1 = (line-1)*patchx, line*patchx
            if x1 > reservoir_shape[0]: # reservoirs[mosaic_n][1][0] :
                x0, x1 = reservoir_shape[0]-patchx, reservoir_shape[0]

            mosaic[x0:x1 , y0:y1, 2 ] = patch_img[:,:,2]

    name = output_name

    cv2.imwrite(name, mosaic)
    print('assembled predition outputs :')

    plt.imshow(mosaic)
    plt.show()
   # return  #mosaic

In [ ]:
def find_reser_aeras(im):

    labels = measure.label(im, connectivity=1)
    props = measure.regionprops(labels)
    area = [prop.area for prop in props]

    return area,labels


def delete_holes(im,area,labels,threshold):
    smallclouds = [t[0] for t in filter(lambda a: a[1] < threshold, enumerate(area, 1))]
    for label in smallclouds:
        im[labels == label] = 1

    return im

def delete_water_bodies(im,area,labels,threshold):
    smallclouds = [t[0] for t in filter(lambda a: a[1] < threshold, enumerate(area, 1))]
    for label in smallclouds:
        im[labels == label] = 0
    return im


In [ ]:
def morpho_operation(resolution,mosaic_addr):

    k = int(100/resolution)
    kernel = np.ones((k,k),np.uint8)

    img = cv2.imread(mosaic_addr)[:,:,2]
    maximg = np.max(img)
    img = np.floor(img/190)


    closing = cv2.morphologyEx(img , cv2.MORPH_CLOSE, kernel)

    opening = cv2.morphologyEx(closing , cv2.MORPH_OPEN, kernel)

    newimg = np.zeros((img.shape[0],img.shape[1],3))
    newimg[:,:,2] = opening[:,:]#*100

    return newimg

def rule_based_refinment(newimg,addr_to_save):

    input_img = newimg[:,:,2].astype(int)

    mosaic_wo_small_res = input_img

    area,labels = find_reser_aeras(mosaic_wo_small_res)
    max_area, min_area = max(area),min(area)
    threshold= int(max_area/10)
    refined_1 = delete_water_bodies(mosaic_wo_small_res,area,labels,threshold)

    #********** holes :
    input_img = refined_1
    area,labels = find_reser_aeras(input_img+1)
    max_area, min_area = max(area),min(area)

    threshold= int(max_area/10)
    refined_2 = delete_holes(input_img,area,labels,threshold)


    print('final post processing output:')
    plt.imshow(refined_2*100)
    plt.show()
    cv2.imwrite(addr_to_save,refined_2)


In [ ]:
def morpho_and_rulebased_refinmnets(resolution,mosaic_addr,addr_to_save):

    morpho_refinment = morpho_operation(resolution,mosaic_addr)

    rule_based_refinment(morpho_refinment,addr_to_save)
